In [1]:
import pandas as pd
import pickle

In [4]:
df = pd.read_csv('Spam_SMS_Collection.txt',sep='\t',names=['label','message'])

In [5]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer

In [9]:
# Cleaning the messages
ccorpus = []
ps = PorterStemmer()

for i in range(0,df.shape[0]):

  # Cleaning special character from the message
  message = re.sub(pattern='[^a-zA-Z]', repl=' ', string=df.message[i])

  # Converting the entire message into lower case
  message = message.lower()

  # Tokenizing the review by words
  words = message.split()

  # Removing the stop words
  words = [word for word in words if word not in set(stopwords.words('english'))]

  # Stemming the words
  words = [ps.stem(word) for word in words]

  # Joining the stemmed words
  message = ' '.join(words)

  # Building a corpus of messages
  corpus.append(message)

In [13]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray()

In [17]:
Y = pd.get_dummies(df['label'])
Y.head()

,ham,spam
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [18]:
Y  = Y.iloc[:,1].values

In [21]:
# Creating a pickle file for the CountVectorizer
pickle.dump(cv,open('cv.pkl','wb'))

In [22]:
# Model Building
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [24]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [25]:
pickle.dump(classifier,open('model.pkl','wb'))

In [26]:
from sklearn.metrics import confusion_matrix,accuracy_score
pred = classifier.predict(X_test)
cm =confusion_matrix(Y_test,pred)
ac = accuracy_score(Y_test,pred)

In [27]:
cm

array([[946,  20],
       [  6, 143]], dtype=int64)

In [28]:
ac

0.9766816143497757